In [1]:
%%time
# Compiling HexState
from Numba_hex_class import HexState, _simulate
_simulate(10)

Wall time: 11.9 s


In [2]:
from random import randint
from math import sqrt, log
from numba import njit, prange
from numba import int64, deferred_type, optional, types, float64, typed
from numba.experimental import jitclass
import numpy as np


In [3]:
%%time
@njit(parallel=True)
def run_sims():
    for x0 in prange(100):
        _simulate(100)
        
run_sims()
# Multi processed
%time run_sims()

# Not multi-processed
%time _simulate(10000)

Wall time: 718 ms
Wall time: 1.19 s
Wall time: 9.32 s


In [146]:
#NODE CLASS
Node_type = deferred_type()

spec = (

    ('parent', optional(Node_type)),
    ('move', int64),
    ('children', int64[:]),

    ('N', int64),
    ('Q', int64),
    ('N_rave', int64),
    ('Q_rave', int64),

)


@jitclass(spec)
class Node:
    def __init__(self, parent, move):
        self.parent = parent
        self.move = move
        self.children = np.zeros(0, dtype=np.int64)

        self.N = 0
        self.Q = 0
        self.N_rave = 0
        self.Q_rave = 0

    def value(self):
        if self.N == 0: 
            return float64(1000)

        rave_weight = max(0, 1 - (self.N/300))
        parent = self.parent
        UCT_value = (self.Q/self.N) + 0.5 * sqrt(2 * log(parent.get_N()/self.N))
        if self.N_rave != 0:
            rave_value = self.Q_rave/self.N_rave
        else:
            rave_value = 0
        value = (1 - rave_weight)*UCT_value + rave_weight*rave_value
        # print(rave_value, value)
        print(rave_weight, parent.get_N())
        return float64(value)

    def set_children(self, new_children):
        self.children = np.append(self.children, new_children)

    def get_N(self):
        return self.N

    def get_children(self):
        return self.children

    def get_move(self):
        return self.move
    
    def get_parent(self):
        return self.parent

    def addon_rave(self, q, n):
        self.Q_rave += q
        self.N_rave += n

    def addon_mcts(self, q, n):
        self.Q += q
        self.N += n
    
    def get_stats(self):
        print(f'N:{self.N}, Q:{self.Q}, Q_r:{self.Q_rave}, N_r:{self.N_rave}, M:{self.move}')
        print(self.value())


Node_type.define(Node.class_type.instance_type)


In [147]:
#EXPAND
@njit
def expand(parent: Node, state, mem, mem_address):
    if state.winner() != 0:
        return (0, mem_address)

    possible_moves = state.possible_moves()
    children = np.zeros_like(possible_moves)
        
    for index, move in enumerate(possible_moves):
        child_node = Node(parent, move)
        mem_address += 1
        mem[mem_address] = child_node
        children[index] = mem_address

    parent.set_children(children)
    return (1, mem_address)


In [148]:
#lead node
@njit
def leaf_node(root_node : Node, root_state, mem, mem_addrs):
    
    node = root_node
    state = root_state
    
    while node.get_children().size != 0:
        # print('readched herre')
        
        array = np.zeros(0, np.int64)
        benchmark = float64(-1000)
        
        for child_mem_addrs in node.get_children():
            child = mem[child_mem_addrs]
            value_of_child = child.value()
            
            if value_of_child > benchmark:
                benchmark = value_of_child
                array = np.zeros(0, np.int64)
                array = np.append(array, child_mem_addrs)
                
            elif value_of_child == benchmark:
                array = np.append(array, child_mem_addrs)
        
        selected_index_from_array = randint(0, array.size-1)
        selected_node  = mem[array[selected_index_from_array]]
        node = selected_node
        move = node.get_move()
        state.step(move)

        if node.get_N() == 0:
            return (node, state, mem_addrs)

    bool_expand, mem_addrs_new = expand(node, state,mem, mem_addrs)
    if bool_expand:
        children_addrs = node.get_children()
        child_index = randint(0, children_addrs.size-1)
        child_addrs = children_addrs[child_index]
        node = mem[child_addrs]
        state.step(node.get_move())

    return (node, state, mem_addrs_new)


In [149]:
#rollout
@njit
def rollout(state):
    # print('reached rollout')
    moves = state.possible_moves()
    while state.winner() == 0:
        move_index = randint(0, moves.size-1)
        move = moves[move_index]
        state.step(move)
        moves = np.delete(moves, move_index)

    blk_rave_pieces = np.zeros(0, dtype=np.int64)
    wht_rave_pieces = np.zeros(0, dtype=np.int64)

    board = state.get_board()
    blk_rave_pieces = np.append(blk_rave_pieces,np.where(board == 1)[0] )
    wht_rave_pieces = np.append(wht_rave_pieces,np.where(board == -1)[0] )
    # print(state.winner())

    return (state.winner(), blk_rave_pieces, wht_rave_pieces)


In [150]:
#BACKUP
@njit
def backup(outcome, turn, node, blk_rave_pieces, wht_rave_pieces, mem ):
    
    if outcome == turn:
        reward = -1
    else:
        reward = 1
        
    while node is not None:
        if turn == 1:
            for child_addrs in node.get_children():
                child  = mem[child_addrs]
                child_move = child.get_move()
                if child_move in blk_rave_pieces:
                    child.addon_rave(-reward, 1)
        else:
            for child_addrs in node.get_children():
                child  = mem[child_addrs]
                child_move = child.get_move()
                if child_move in wht_rave_pieces:
                    child.addon_rave(-reward, 1)
                    
        node.addon_mcts(reward, 1)

        turn *= -1
        reward *= -1
        node = node.get_parent()
    # print('backup ended')

In [151]:
#fetch_best_move
@njit
def fetch_best_move(state, limit):
    # print('STARYED')
    
    memory = {}
    memory_address = 0
    
    root_state = state.copy()
    root_node = Node(None, 1000)
    memory[memory_address] = root_node
    
    num_simulation = 0
    
    while num_simulation < limit:
        
        state_copy = root_state.copy()
        
        node, new_state, memory_address = leaf_node(root_node,state_copy,memory, memory_address)
        turn = new_state.get_to_play()
        winner, blk_rave_pieces, wht_rave_pieces = rollout(new_state)
        
        backup(winner, turn, node, blk_rave_pieces, wht_rave_pieces, memory)
        # print('Reacheed outidde')
        num_simulation +=1
        
    i  = 0
    while i<36:
        n = memory[i]
        i +=1
        memory[i].get_stats()

    

In [152]:
#SETTING UP EMPTY BOARD
parent_type = (types.int64, types.int64)
rank_type = (types.int64, types.int64)
groups_type = (types.int64, types.int64[:])
size = 6

brd = np.zeros(size**2, dtype=np.int64)
to_play = 1
EDGE_START = 1000
EDGE_FINISH = -1000
blk_parent = typed.Dict.empty(*parent_type)
blk_rank = typed.Dict.empty(*rank_type)
blk_groups = typed.Dict.empty(*groups_type)
wht_parent = typed.Dict.empty(*parent_type)
wht_rank = typed.Dict.empty(*rank_type)
wht_groups = typed.Dict.empty(*groups_type)


board = HexState(size, brd, to_play, EDGE_START, EDGE_FINISH, blk_parent, blk_rank, blk_groups, wht_parent, wht_rank, wht_groups)

In [153]:
fetch_best_move(board, 1)

N:0, Q:0, Q_r:0, N_r:0, M:0
1000.0
N:0, Q:0, Q_r:0, N_r:0, M:1
1000.0
N:0, Q:0, Q_r:0, N_r:0, M:2
1000.0
N:0, Q:0, Q_r:0, N_r:0, M:3
1000.0
N:0, Q:0, Q_r:-1, N_r:1, M:4
1000.0
N:0, Q:0, Q_r:0, N_r:0, M:5
1000.0
N:0, Q:0, Q_r:-1, N_r:1, M:6
1000.0
N:0, Q:0, Q_r:0, N_r:0, M:7
1000.0
N:0, Q:0, Q_r:-1, N_r:1, M:8
1000.0
N:0, Q:0, Q_r:-1, N_r:1, M:9
1000.0
N:1, Q:-1, Q_r:-1, N_r:1, M:10
0.9966666666666667 1
-1.0
N:0, Q:0, Q_r:0, N_r:0, M:11
1000.0
N:0, Q:0, Q_r:-1, N_r:1, M:12
1000.0
N:0, Q:0, Q_r:0, N_r:0, M:13
1000.0
N:0, Q:0, Q_r:-1, N_r:1, M:14
1000.0
N:0, Q:0, Q_r:-1, N_r:1, M:15
1000.0
N:0, Q:0, Q_r:0, N_r:0, M:16
1000.0
N:0, Q:0, Q_r:-1, N_r:1, M:17
1000.0
N:0, Q:0, Q_r:0, N_r:0, M:18
1000.0
N:0, Q:0, Q_r:-1, N_r:1, M:19
1000.0
N:0, Q:0, Q_r:0, N_r:0, M:20
1000.0
N:0, Q:0, Q_r:0, N_r:0, M:21
1000.0
N:0, Q:0, Q_r:-1, N_r:1, M:22
1000.0
N:0, Q:0, Q_r:-1, N_r:1, M:23
1000.0
N:0, Q:0, Q_r:-1, N_r:1, M:24
1000.0
N:0, Q:0, Q_r:0, N_r:0, M:25
1000.0
N:0, Q:0, Q_r:0, N_r:0, M:26
1000.0
N:0, 

In [154]:
%time a = fetch_best_move(board, 100)

0.9966666666666667 1
0.9966666666666667 2
0.9966666666666667 2
0.9966666666666667 3
0.9966666666666667 3
0.9966666666666667 3
0.9966666666666667 4
0.9966666666666667 4
0.9966666666666667 4
0.9966666666666667 4
0.9966666666666667 5
0.9966666666666667 5
0.9966666666666667 5
0.9966666666666667 5
0.9966666666666667 5
0.9966666666666667 6
0.9966666666666667 6
0.9966666666666667 6
0.9966666666666667 6
0.9966666666666667 6
0.9966666666666667 6
0.9966666666666667 7
0.9966666666666667 7
0.9966666666666667 7
0.9966666666666667 7
0.9966666666666667 7
0.9966666666666667 7
0.9966666666666667 7
0.9966666666666667 8
0.9966666666666667 8
0.9966666666666667 8
0.9966666666666667 8
0.9966666666666667 8
0.9966666666666667 8
0.9966666666666667 8
0.9966666666666667 8
0.9966666666666667 9
0.9966666666666667 9
0.9966666666666667 9
0.9966666666666667 9
0.9966666666666667 9
0.9966666666666667 9
0.9966666666666667 9
0.9966666666666667 9
0.9966666666666667 9
0.9966666666666667 10
0.9966666666666667 10
0.996666666

In [118]:
%time a = fetch_best_move(board, 1000)

N:1, Q:-1, Q_r:7, N_r:381, M:0
0.02117299804878295
N:1, Q:-1, Q_r:17, N_r:433, M:1
0.041991637058398666
N:1, Q:1, Q_r:18, N_r:410, M:2
0.053284301209058575
N:1, Q:1, Q_r:45, N_r:397, M:3
0.12250049583951873
N:1, Q:1, Q_r:49, N_r:419, M:4
0.1260834940219891
N:1, Q:1, Q_r:44, N_r:484, M:5
0.10013426425414357
N:1, Q:-1, Q_r:13, N_r:401, M:6
0.03517242642447539
N:1, Q:1, Q_r:39, N_r:425, M:7
0.10098702717749473
N:1, Q:1, Q_r:36, N_r:426, M:8
0.09375355576075901
N:33, Q:-1, Q_r:68, N_r:462, M:9
0.27132182635337304
N:1, Q:-1, Q_r:58, N_r:442, M:10
0.13364585070690657
N:1, Q:1, Q_r:52, N_r:414, M:11
0.13471338883326817
N:1, Q:-1, Q_r:39, N_r:389, M:12
0.10278441615879426
N:1, Q:1, Q_r:61, N_r:389, M:13
0.16581783518193044
N:1, Q:-1, Q_r:90, N_r:446, M:14
0.20398261321460023
N:1, Q:-1, Q_r:69, N_r:419, M:15
0.1669904152630393
N:1, Q:-1, Q_r:45, N_r:401, M:16
0.1147069235150822
N:1, Q:1, Q_r:47, N_r:403, M:17
0.1257647627878679
N:1, Q:1, Q_r:65, N_r:407, M:18
0.16870101282089212
N:1, Q:-1, Q_r:

In [61]:
@njit
def test():
    a= 5
    b= 9
    c = 3
    print(c)
    c= 5/9
    print(c)
    return c
duh =  test()
print(duh)

3
0.5555555555555556
0.5555555555555556
